In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import json
import numpy as np
import pandas as pd
import requests

import openpyxl
from openpyxl import load_workbook
import os

from itertools import product

In [2]:
os.getcwd()

'c:\\Users\\DavidKnapp\\OneDrive - Opus Music Group Holdings, LLC\\Desktop\\omg_spotify'

In [3]:
def config_spotify():
    config = '{"cid": "a8a94da4709b43a9a97f09ebe3f7f5c7"\
                    ,"secret": "6e19fc0b10234e4283dd9a02b5f88aa0"}'
    return json.loads(config)

cred_sp = config_spotify()

cid = cred_sp["cid"]
secret = cred_sp["secret"]
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret,requests_timeout = 10)
# client_credentials_manager.get_access_token()
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [4]:
df = pd.read_excel('RawDataTemplate.xlsx',sheet_name="Input Details")

df.head()

c:\Users\DavidKnapp\OneDrive - Opus Music Group Holdings, LLC\Desktop\omg_spotify\spotify_env\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,song_name_api_opus,song_name_opus,artist_name_opus,catalog_name_opus,owned_catalog_opus,input_parameter,country_code,song_id_luminate_opus,isrc_opus,album_name_opus,release_date_opus,metric_luminate,start_week_id,end_week_id
0,Hotel in Sydney,Hotel in Sydney,Hopsin,Hopsin,No,isrc,G1,NaN,TCAEP2021865,No Shame,2017-11-24 00:00:00,StreamingOnDemand,201840,202335
1,Right Here,Right Here,Hopsin,Hopsin,No,isrc,G1,NaN,TCAEP2021877,No Shame,2017-11-24 00:00:00,StreamingOnDemand,201840,202335
2,Twisted,Twisted,Hopsin,Hopsin,No,isrc,G1,NaN,TCAEP2021881,No Shame,2017-11-24 00:00:00,StreamingOnDemand,201840,202335
3,Cute in a Suit (Skit),Cute in a Suit (Skit),Hopsin,Hopsin,No,isrc,G1,NaN,TCAEP2021882,No Shame,2017-11-24 00:00:00,StreamingOnDemand,201840,202335
4,All Your Fault (Remix),All Your Fault (Remix),Hopsin,Hopsin,No,isrc,G1,NaN,TCAEP2021884,No Shame,2017-11-24 00:00:00,StreamingOnDemand,201840,202335


In [5]:
def isrc_spotify_lookup(input_record):
    # get song from row of input file
    input_song = input_record['song_name_api_opus']
    print(input_song)
    # get artists from row of input file
    input_artists = input_record['artist_name_opus'].split(',')
    input_artists = [a.strip() for a in input_artists]
    
    # for a song with multiple artists, create one query for each combination of song title, artist
    query_tuples = list(product([input_song],input_artists))
    sp_query = f'track:{query_tuples[0][0]} artist: {query_tuples[0][1]}'
    # remove apostrophes from query to improve results
    sp_queries = [f'track:{qt[0]} artist: {qt[1]}'.replace("'","") for qt in query_tuples]

    all_matches = list()

    for sq in sp_queries:
        track_results = sp.search(q = sq, type = 'track')
        track_list = track_results['tracks']['items']

        # filter on exact match
        if len(track_list) > 0:
            print(f'At least one match found for query {sq}')
            track_hits = [(t['name'],t['popularity'],t['external_ids']['isrc']) for t in track_list]
            track_hits
            for t in track_hits:
                all_matches.append(t)
        else:
            print(f'No Match Found for query {sq}')
        

    isrc_df = pd.DataFrame(all_matches)
    # break ties if more than one result using highest popularity
    if isrc_df.shape[0] > 0:
        isrc_max_indx = isrc_df[1].idxmax()
        selected_isrc = isrc_df[2][isrc_max_indx]
    else:
        selected_isrc = 'Not Found'
        
    return selected_isrc

isrcs = [isrc_spotify_lookup(row) for idx, row in df.iterrows()]

Hotel in Sydney
At least one match found for query track:Hotel in Sydney artist: Hopsin
Right Here
At least one match found for query track:Right Here artist: Hopsin
Twisted
At least one match found for query track:Twisted artist: Hopsin
Cute in a Suit (Skit)
At least one match found for query track:Cute in a Suit (Skit) artist: Hopsin
All Your Fault (Remix)
At least one match found for query track:All Your Fault (Remix) artist: Hopsin
Money on the Side
At least one match found for query track:Money on the Side artist: Hopsin
I Wouldn't Do That
At least one match found for query track:I Wouldnt Do That artist: Hopsin
Black Sheep
At least one match found for query track:Black Sheep artist: Hopsin
I Must Be On Something
No Match Found for query track:I Must Be On Something artist: Hopsin
Tell'em Who You Got It From
At least one match found for query track:Tellem Who You Got It From artist: Hopsin
The Purge
At least one match found for query track:The Purge artist: Hopsin
Happy Ending
At 

In [6]:
df['isrc_opus'] = isrcs
df.to_csv('./input_data_isrc.csv')